<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/trainingPreTrainedYoloV5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import re
import json
import datetime
import pandas as pd
from PIL import Image, ImageDraw
from io import BytesIO
import yaml
import os
try:
    from roboflow import Roboflow
except ImportError as e:
    !pip install roboflow

try:
  from ultralytics import YOLO
except ImportError as e:
  !pip install ultralytics

from ultralytics import YOLO

;

''

In [4]:
ADD THESE FILES TO MY GOOGLE DRIVE AND LOAD THAT WAY
# Load player detection dataset from Roboflow
rf = Roboflow(api_key="e4YWipTNVce3CfUmX7fc")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(11)
dataset = version.download("yolov9")
# Load ball detection dataset from Roboflow
project = rf.workspace("roboflow-jvuqo").project("football-ball-detection-rejhg")
version = project.version(4)
dataset = version.download("yolov9")

# Function to update state in a yaml file
def set_state(yamlFileLoc,absPath):
    with open(yamlFileLoc) as f:
        doc = yaml.safe_load(f)

    doc['test'] = absPath+"/test/images"
    doc['train'] = absPath+"/train/images"
    doc['val'] = absPath+"/valid/images"
    with open(yamlFileLoc, 'w') as f:
        yaml.dump(doc, f)

#Find absolute path of dataset directories
ballDetectionPath = os.path.abspath('football-ball-detection-4')
playerDetectionPath = os.path.abspath('football-players-detection-11')

# Update YAML files to include full paths to datasets
set_state('/content/football-players-detection-11/data.yaml',playerDetectionPath)
set_state('/content/football-ball-detection-4/data.yaml',ballDetectionPath)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-11 in yolov9:: 100%|██████████| 756/756 [00:00<00:00, 2727.01it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-ball-detection-4 in yolov9:: 100%|██████████| 9908/9908 [00:01<00:00, 6604.59it/s]


In [ ]:
url = 'https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/retrainingDataset/_annotations.coco'

resp = requests.get(url)
data = json.loads(resp.text)
json_formatted_str = json.dumps(data, indent=2)

In [ ]:
with open('trainData.yaml', 'w') as yaml_file:
    yaml.dump(resp.json(), yaml_file)

In [ ]:
imageID = 18
particularImage = [x for x in data['images'] if x['id'] == imageID]
boundingBoxes = [x for x in data['annotations'] if x['image_id'] == imageID]


In [ ]:
imageURL = "https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/retrainingDataset/"+particularImage[0]['file_name']+"?raw=true"
imageURL

response = requests.get(imageURL)
img = Image.open(BytesIO(response.content))
img1 = ImageDraw.Draw(img)


#display(img)

In [ ]:
for box in boundingBoxes:
  shape = [(box['bbox'][0], box['bbox'][1]), (box['bbox'][0]+box['bbox'][2], box['bbox'][1]+box['bbox'][3])]
  if box['category_id'] == 3:
    img1.rectangle(shape, outline ="red")
  if box['category_id'] == 1:
    img1.rectangle(shape, outline ="blue")
display(img)

In [ ]:


# Load a COCO-pretrained YOLOv8n model
#model = YOLO("/content/runs/detect/train3/weights/best.pt")
model = YOLO("yolov8l.pt")
# Train the model on the COCO8 example dataset for 100 epochs

results = model.train(data="/content/football-players-detection-11/data.yaml", epochs=300, imgsz=640)

100%|██████████| 83.7M/83.7M [00:00<00:00, 468MB/s]


Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/football-players-detection-11/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

100%|██████████| 755k/755k [00:00<00:00, 103MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 239MB/s]


...

      6/300      11.3G      1.292     0.7907     0.8568        287        640: 100%|██████████| 19/19 [00:14<00:00,  1.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.91it/s]

                   all         49       1174      0.194      0.185     0.0768     0.0295



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      7/300        11G      1.171     0.7233     0.8417        259        640: 100%|██████████| 19/19 [00:14<00:00,  1.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.95it/s]

                   all         49       1174      0.659      0.564      0.608      0.358
